# Building a RAG Agent with CrewAI

In this lesson, you will build a RAG agent with `CrewAI`. You will do that by integrating `RagTool` from `crewai_tools` with a `CrewAI` agent. `RagTool` provides a way to create and query knowledge bases from various data sources, and allows the agent to access specialized context. In this lesson, you will provide the RAG tool a pdf file containing details about insurance coverage provided by a private health insurer. By the end of the lesson, you will build an insurer agent specialized in answering queries related to health benefits. In the next lessons, you will wrap this agent in an ACP server and make it interact with other ACP agents.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

## 3.1. Import Libraries

In [18]:
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

In [19]:
import warnings
warnings.filterwarnings('ignore')

## 3.2. Define the Agent's Large Language Model

You'll now define the large language model that you will use for your CrewAI agent. `max_tokens`: maximum number of tokens the model can generate in a single response.

**Note**: If you will define this model locally, it requires that you define the API key in a **.env** file as follows:
```
# Required
OPENAI_API_KEY=sk-...

# Optional
OPENAI_API_BASE=<custom-base-url>
OPENAI_ORGANIZATION=<your-org-id>
```

In [20]:
llm = LLM(model="openai/gpt-4", max_tokens=1024)

## 3.3. Define the RAG Tool

For the RAG tool, you can define the model provider and the embedding model in a configuration Python dictionary. You can also define the details of your vector database. If you don't specify the vector database, the RagTool will use Chroma (ChromaDB) as the default vector database in local/in-memory mode.

In [21]:
config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}

You can then pass the `config` to the `RagTool`, and then specify the data source for which the knowledge base will be constructed. When embedding your data, the `RagTool` chunks your document into chunks and create an embedding vector for each chunk. You can specify the chunk size (`chunk_size`: number of characters) and how many characters overlap between consecutive chunks (`chunk_overlap`). You can also use the default behavior.

In [22]:
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("../data/Handbook of Information for Students 2025-26.pdf", data_type="pdf_file")

In [23]:
rag_tool

RagTool(name='Knowledge base', description="Tool Name: Knowledge base\nTool Arguments: {'query': {'description': None, 'type': 'str'}}\nTool Description: A knowledge base that can be used to answer questions.", args_schema=<class 'abc.RagToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x7f472f21b2e0>, result_as_answer=False, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x7f47221eaa90>, summarize=False), config={'llm': {'provider': 'openai', 'config': {'model': 'gpt-4'}}, 'embedding_model': {'provider': 'openai', 'config': {'model': 'text-embedding-ada-002'}}})

## 3.4. Define the Counselling Agent

Now that you have the `rag_tool` defined, you define the CrewAI agent that can assist with insurance coverage queries.

In [24]:
counselling_agent = Agent(
    role="Senior College Counsellor Agent", 
    goal="Determine the best academic practices for a student in the PGDM Program",
    backstory="You are an expert counselling agent designed to assist with academic queries",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[rag_tool], 
    max_retry_limit=5
)

## 3.5. Define the Agent Task

Let's now test the counselling agent. For that, you need to define the agent task and pass to it the query and the agent.

In [25]:
task1 = Task(
        description='What is the criteria for receiving the Deans List Award?',
        expected_output = "A comprehensive response as to the users question",
        agent=counselling_agent
)

## 3.6. Run the Counselling Agent

To run the agent, you need to pass the agent and the task to a Crew object that you can run using the `kickoff` method.

In [26]:
crew = Crew(agents=[counselling_agent], tasks=[task1], verbose=True)
task_output = crew.kickoff()
print(task_output) 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 17a5f323-f65d-4bf2-9a53-4d0354abefe5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 45bf5845-fc1d-40c7-94bf-de607ecce148
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 45bf5845-fc1d-40c7-94bf-de607ecce148
       Status: Executing Task...
    └── 🤖 Agent: Senior College Counsellor Agent
            Status: In Progress

# Agent: Senior College Counsellor Agent
## Task: What is the criteria for receiving the Deans List Award?


🤖 Agent: Senior College Counsellor Agent
    Status: In Progress

...

🚀 Crew: crew
└── 📋 Task: 45bf5845-fc1d-40c7-94bf-de607ecce148
       Status: Executing Task...
    └── 🤖 Agent: Senior College Counsellor Agent
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 45bf5845-fc1d-40c7-94bf-de607ecce148
       Assigned to: Senior College Counsellor Agent
       Status: ✅ Completed
    └── 🤖 Agent: Senior College Counsellor Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 45bf5845-fc1d-40c7-94bf-de607ecce148                                                                     │
│  Agent: Senior College Counsellor Agent                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 17a5f323-f65d-4bf2-9a53-4d0354abefe5                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The criteria for receiving the Dean's List Award in the PGDM Program are as follows:

  - For the first year of the program:
    1. The student should secure a CGPA of 7.0 or higher after the completion of the first year of the program.
    2. The student must not have secured a ‘D’ or ‘F’ in any course or ‘NS’ in a compulsory workshop.
    3. The student who has not appeared for any make-up examination.
    4. The student must not have been penalized for resorting to unfair means in completing any of the evaluation components.

  - For the second year of the program:
    1. The student should secure a CGPA of 7.0 or higher at the end of the second year of the program.
    2. The student should secure a CGPA of 7.0 or higher after the completion of the first year of the program.
    3. The student must not have secured a ‘D’ or ‘F’ in any course or an ‘NS’ in a compulsory workshop.
    4. The student who has not appeared for any make-up examination.
    5. The student must not have bee

## 3.7. Resources

- [CrewAI Agents](https://docs.crewai.com/concepts/agents)
- [CrewAI Tasks](https://docs.crewai.com/concepts/tasks)
- [CrewAI RagTool](https://docs.crewai.com/tools/ai-ml/ragtool)
- [Short course on Multi Agents with CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>